In [84]:
#Credit https://mccormickml.com/2016/04/12/googles-pretrained-word2vec-model-in-python/
import inflect
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import SimpleRNN, Dense
from keras.optimizers import Adam
from gensim.models.keyedvectors import KeyedVectors
from sklearn.model_selection import train_test_split

In [85]:
from google.colab import drive
drive.mount('/mntDrive') #, force_remount=True)

Drive already mounted at /mntDrive; to attempt to forcibly remount, call drive.mount("/mntDrive", force_remount=True).


In [86]:
# Initialize the inflect engine
p = inflect.engine()

In [87]:
# Generate number words from 1 to n as an example
N = 20#maximum number included in sequences
n = 11#number of instances
s_size = 5
numbers = list(range(1, N+1))#numbers 1 to 20, inclusive
number_words = [p.number_to_words(num) for num in numbers]

# Create mappings
word_to_num = {word: num for word, num in zip(number_words, numbers)}
num_to_word = {num: word for word, num in word_to_num.items()}

# Generate dataset sequences
sequences = []
for i in range(len(numbers) - (2*s_size)+1):#i from 0 to 10, inclusive
    input_seq = number_words[i:i+s_size]#i from 0 to 4 up to from 10 to 14, inclusive
    target_seq = numbers[i+s_size:i+(2*s_size)]#from 5 to 9 up to from 15 to 19, inclusive
    sequences.append((input_seq, target_seq))


In [88]:
print(len(sequences[1][0]))

5


In [89]:
# Load Google's pre-trained Word2Vec model.
path = "/mntDrive/MyDrive/DSIWorkshop/DLWorkshop/GoogleNews-vectors-negative300.bin"
word2vec_model = KeyedVectors.load_word2vec_format(path, binary=True)


In [90]:
def word_to_embedding(word):
    return word2vec_model[word]

In [103]:
# Define the RNN model
model = Sequential()
model.add(SimpleRNN(50, input_shape=(s_size, 300), return_sequences=True))#Google's Word2Vec uses 300 features.
model.add(Dense(1))
adam = Adam(learning_rate=0.01)
model.compile(optimizer=adam, loss='mse')

In [104]:
# Prepare training data
X = np.array([np.array([word_to_embedding(word) for word in seq]) for seq, _ in sequences])
y = np.array([np.array(target) for _, target in sequences])
test_prop = 0.25
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=1,test_size=test_prop,shuffle=True)

In [105]:
# Train the model
model.fit(X_train, y_train, epochs=1000)


Epoch 1/1000
1/1 [==============================] - 1s 1s/step - loss: 208.9955
Epoch 2/1000
1/1 [==============================] - 0s 13ms/step - loss: 143.2561
Epoch 3/1000
1/1 [==============================] - 0s 13ms/step - loss: 99.4594
Epoch 4/1000
1/1 [==============================] - 0s 10ms/step - loss: 73.9397
Epoch 5/1000
1/1 [==============================] - 0s 11ms/step - loss: 58.6314
Epoch 6/1000
1/1 [==============================] - 0s 14ms/step - loss: 48.4459
Epoch 7/1000
1/1 [==============================] - 0s 15ms/step - loss: 40.8636
Epoch 8/1000
1/1 [==============================] - 0s 16ms/step - loss: 34.8025
Epoch 9/1000
1/1 [==============================] - 0s 15ms/step - loss: 29.9240
Epoch 10/1000
1/1 [==============================] - 0s 14ms/step - loss: 25.9112
Epoch 11/1000
1/1 [==============================] - 0s 14ms/step - loss: 22.5419
Epoch 12/1000
1/1 [==============================] - 0s 13ms/step - loss: 19.7197
Epoch 13/1000
1/1 [======

In [106]:
y_test_ = np.rint(model.predict(X_test))
y_test_ = y_test_.reshape(y_test.shape)
acc1 = np.sum(y_test.flatten() == y_test_.flatten())/len(y_test.flatten())*100
print('Accuraccy:',acc1,'%')
print(y_test[:5])
print(y_test_[:5])


1/1 [==============================] - 0s 121ms/step
Accuraccy: 60.0 %
[[ 8  9 10 11 12]
 [ 9 10 11 12 13]
 [10 11 12 13 14]]
[[ 9.  9. 10. 11. 12.]
 [10. 11. 12. 12. 13.]
 [11. 12. 12. 13. 14.]]
